In [1]:
import tensorflow as tf

# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
#
# The value returned by the constructor represents the output
# of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix
# multiplication.
product = tf.matmul(matrix1, matrix2)

In [2]:
# Launch the default graph.
sess = tf.Session()

# To run the matmul op we call the session 'run()' method, passing 'product'
# which represents the output of the matmul op.  This indicates to the call
# that we want to get the output of the matmul op back.
#
# All inputs needed by the op are run automatically by the session.  They
# typically are run in parallel.
#
# The call 'run(product)' thus causes the execution of three ops in the
# graph: the two constants and matmul.
#
# The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# Close the Session when we're done.
sess.close()

[[ 12.]]


In [3]:
import numpy as np
import pandas as pd
import scipy
from scipy.sparse import csr_matrix, diags, eye
from sklearn.cluster import SpectralClustering
from matplotlib import pyplot as plt
import pylab

plt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = (16.0, 10.0)

%matplotlib inline

In [4]:
df = pd.read_csv("./facebook_combined.txt.gz", compression='infer', sep=" ", header=None, names=["a", "b"])
e = len(df)
v = len(np.unique(df["a"]))
v = max(np.max(np.unique(df["a"])), np.max(np.unique(df["b"]))) + 1
print(e,v)
facebook2 = csr_matrix((np.ones(e), (df.a, df.b)), shape=(v,v))
facebook2

88234 4039


<4039x4039 sparse matrix of type '<class 'numpy.float64'>'
	with 88234 stored elements in Compressed Sparse Row format>

In [8]:
sess = tf.InteractiveSession()

g = tf.SparseTensor(indices=[[a,b] for (a,b) in df.to_records(index=False)], values = np.ones(e), shape=[v,v])
g1 = tf.sparse_tensor_to_dense(g, 0.0)

%time s = tf.add(g1, g1)

%time sr = s.eval()

%time m = tf.sparse_tensor_dense_matmul(g, g1)

%time mr = m.eval()

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fc5eaefbf98>>
Traceback (most recent call last):
  File "/home/yannick/bin/anaconda3/envs/tf/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 140, in __del__
    self.close()
  File "/home/yannick/bin/anaconda3/envs/tf/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 905, in close
    self._default_session.__exit__(None, None, None)
  File "/home/yannick/bin/anaconda3/envs/tf/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/home/yannick/bin/anaconda3/envs/tf/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 3215, in get_controller
    assert self.stack[-1] is default
AssertionError: 


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 364 µs
CPU times: user 168 ms, sys: 16 ms, total: 184 ms
Wall time: 247 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.04 ms
CPU times: user 492 ms, sys: 28 ms, total: 520 ms
Wall time: 498 ms


In [14]:
%time sr_np = facebook2 + facebook2
%time mr_np = facebook2.dot(facebook2)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.38 ms
CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 24.7 ms


In [15]:
fd = facebook2.todense()
%time sr_np_d = fd + fd
%time mr_np_d = fd.dot(fd)

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 34 ms
CPU times: user 5.91 s, sys: 0 ns, total: 5.91 s
Wall time: 1.63 s
